<a href="https://colab.research.google.com/github/kenclements/praxis/blob/main/practice2-11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Jan 29 01:26:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.6 MB/s eta 0:00:00


In [42]:
import pandas as pd
import tensorflow as tf
import keras
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, BertForNextSentencePrediction, pipeline, BertForQuestionAnswering, BertForTokenClassification, BertForSequenceClassification, DistilBertTokenizerFast, DataCollatorWithPadding
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [5]:
#email = pd.read_csv('../../../../../../data/r4.2/email.csv')
#file = pd.read_csv('/content/drive/MyDrive/Praxis/data/file.csv')
#http = pd.read_csv('../../../../../../data/r4.2/http.csv')
#logon = pd.read_csv('../../../../../../data/r4.2/logon.csv')
#device = pd.read_csv('../../../../../../data/r4.2/device.csv')
#tweets = pd.read_csv('/content/drive/MyDrive/Praxis/code/tweets.csv')
#enron = pd.read_csv('../../../../../../data/praxis/ENRON/emails_enron.csv')


In [6]:
#files = [file,email,http,logon,device]

In [7]:
#for x in files:
#    print(x.shape)
#file.shape

In [8]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [9]:
from datasets import load_metric, Dataset

In [10]:
snips_file = pd.read_csv('/content/drive/MyDrive/Praxis/data/train.csv')

In [11]:
snips_file

,text,slot,intent
0,listen to westbam alumb allergic on google music,O O B-artist O B-album O B-service I-service,PlayMusic
1,add step to me to the 50 clásicos playlist,O B-entity_name I-entity_name I-entity_name O ...,AddToPlaylist
2,i give this current textbook a rating value of...,O O O B-object_select B-object_type O O O O B-...,RateBook
3,play the song little robin redbreast,O O B-music_item B-track I-track I-track,PlayMusic
4,please add iris dement to my playlist this is ...,O O B-artist I-artist O B-playlist_owner O B-p...,AddToPlaylist
...,...,...,...
13079,i want to eat choucroute at a brasserie for 8,O O O O B-served_dish O O B-restaurant_type O ...,BookRestaurant
13080,play funky heavy bluesy,O B-playlist I-playlist I-playlist,PlayMusic
13081,rate the current album 2 points out of 6,O O B-object_select B-object_type B-rating_val...,RateBook
13082,go to the photograph the inflated tear,O O O B-object_type B-object_name I-object_nam...,SearchCreativeWork


In [12]:
snips_file['intent'].size

13084

In [13]:
utterances = list(snips_file['text'])

In [14]:
tokenized_utterances = []
for j in utterances:
  k = j.split(" ")
  tokenized_utterances.append(k)

In [15]:
tokenized_utterances[0]

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']

In [16]:
labels_for_tokens = []
for j in list(snips_file['slot']) :
  k = j.split(" ")
  labels_for_tokens.append(k)

In [17]:
labels_for_tokens[3]

['O', 'O', 'B-music_item', 'B-track', 'I-track', 'I-track']

In [18]:
print(utterances[0])

listen to westbam alumb allergic on google music


In [19]:
sequence_labels = list(snips_file['intent'])

In [20]:
sequence_labels[0]

'PlayMusic'

In [21]:
len(sequence_labels), len(utterances), len(tokenized_utterances), len(labels_for_tokens)

(13084, 13084, 13084, 13084)

In [22]:
unique_sequence_labels = list(set(sequence_labels))

In [23]:
unique_sequence_labels

['RateBook',
 'BookRestaurant',
 'AddToPlaylist',
 'GetWeather',
 'SearchScreeningEvent',
 'PlayMusic',
 'SearchCreativeWork']

In [24]:
sequence_labels = [unique_sequence_labels.index(l) for l in sequence_labels]

In [25]:
from functools import reduce

In [26]:
unique_token_labels = list(set(reduce(lambda x, y: x + y, labels_for_tokens)))

In [27]:
labels_for_tokens = [[unique_token_labels.index(l) for l in j] for j in labels_for_tokens]

In [28]:
#unique_token_labels

In [46]:
from datasets import Dataset
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizerFast, DataCollatorWithPadding 


In [30]:
snips_dataset = Dataset.from_dict(
    dict(
         utterance = utterances,
         label=sequence_labels,
         tokens = tokenized_utterances,
         token_labels = labels_for_tokens

    )
)
snips_dataset = snips_dataset.train_test_split(test_size=.2)

In [31]:
snips_dataset['train'][0]

{'utterance': 'where can i find the show game of death ii',
 'label': 6,
 'tokens': ['where',
  'can',
  'i',
  'find',
  'the',
  'show',
  'game',
  'of',
  'death',
  'ii'],
 'token_labels': [34, 34, 34, 34, 34, 51, 71, 27, 27, 27]}

In [34]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [35]:
def preprocess_function(examples):
  return tokenizer(examples["utterance"], truncation=True)

In [36]:
seq_clf_tokenized_snips = snips_dataset.map(preprocess_function, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [37]:
seq_clf_tokenized_snips['train'][0]

{'utterance': 'where can i find the show game of death ii',
 'label': 6,
 'tokens': ['where',
  'can',
  'i',
  'find',
  'the',
  'show',
  'game',
  'of',
  'death',
  'ii'],
 'token_labels': [34, 34, 34, 34, 34, 51, 71, 27, 27, 27],
 'input_ids': [101,
  2073,
  2064,
  1045,
  2424,
  1996,
  2265,
  2208,
  1997,
  2331,
  2462,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [43]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [47]:
sequence_clf_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(unique_sequence_labels))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [48]:
sequence_clf_model.config.id2label = {i: l for i, l in enumerate(unique_sequence_labels)}

In [49]:
sequence_clf_model.config.id2label

{0: 'RateBook',
 1: 'BookRestaurant',
 2: 'AddToPlaylist',
 3: 'GetWeather',
 4: 'SearchScreeningEvent',
 5: 'PlayMusic',
 6: 'SearchCreativeWork'}

In [50]:
metric = load_metric('accuracy')

<ipython-input-50-343a7ea47872>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [51]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=labels)



In [52]:
epochs = 2

In [53]:
training_args = TrainingArguments(
    output_dir = "./snips_clf/results",
    num_train_epochs = epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,

    warmup_steps = len(seq_clf_tokenized_snips['train']) // 5,
    weight_decay = .05,

    logging_steps=1,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

In [54]:
trainer = Trainer(
    model = sequence_clf_model,
    args = training_args,
    train_dataset=seq_clf_tokenized_snips['train'],
    eval_dataset=seq_clf_tokenized_snips['test'],
    #compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [55]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 1.9527678489685059,
 'eval_runtime': 4.7777,
 'eval_samples_per_second': 547.755,
 'eval_steps_per_second': 17.163}

In [56]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656
  Number of trainable parameters = 66958855
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss
1,0.230400,0.243052
2,0.744300,0.070894


***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.


Saving model checkpoint to ./snips_clf/results/checkpoint-328
Configuration saved in ./snips_clf/results/checkpoint-328/config.json
Model weights saved in ./snips_clf/results/checkpoint-328/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to ./snips_clf/results/checkpoint-656
Configuration saved in ./snips_clf/results/checkpoint-656/config.json
Model weights saved in ./snips_clf/results/checkpoint-656/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./snips_clf/results/checkpoint-656 (score: 0.07089367508888245).


TrainOutput(global_step=656, training_loss=0.7372562988269392, metrics={'train_runtime': 64.3484, 'train_samples_per_second': 325.323, 'train_steps_per_second': 10.194, 'total_flos': 116624657480508.0, 'train_loss': 0.7372562988269392, 'epoch': 2.0})